# Permuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [2]:
# configurations
data_dir = "./MNIST_data"
n_steps = 28*28
input_dims = 1
n_classes = 10 

# model config
cell_type = "LSTM"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20] * 9
dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = batch_size * 60000
testing_step = 5000
display_step = 100

# permutation seed 
seed = 92916

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
if 'seed' in globals():
    # randomly permute np.arange(n_steps), seed is the state, if the state is the same, the result is the same 
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print ("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)

# evaluation model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with LSTM cells
Building layer: multi_dRNN_dilation_1, input length: 784, dilation rate: 1, input dim: 1.
=====> Input length for sub-RNN: 784
Building layer: multi_dRNN_dilation_2, input length: 784, dilation rate: 2, input dim: 20.
=====> Input length for sub-RNN: 392
Building layer: multi_dRNN_dilation_4, input length: 784, dilation rate: 4, input dim: 20.
=====> Input length for sub-RNN: 196
Building layer: multi_dRNN_dilation_8, input length: 784, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 98
Building layer: multi_dRNN_dilation_16, input length: 784, dilation rate: 16, input dim: 20.
=====> Input length for sub-RNN: 49
Building layer: multi_dRNN_dilation_32, input length: 784, dilation rate: 32, input dim: 20.
=====> 16 time points need to be padded. 
=====> Input length for sub-RNN: 25
Building layer: multi_dRNN_dilation_64, input length: 784, dilation rate: 64, input dim: 20.
=====> 48 time points need to be padded. 
=====> Input length

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
step = 0
train_results = []
validation_results = []
test_results = []

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)    
    batch_x = batch_x[:, idx_permute]
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print ("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # permute the data
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print ("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_)) 
    step += 1

Iter 100, Minibatch Loss: 2.246158, Training Accuracy: 0.164062
Iter 200, Minibatch Loss: 2.189928, Training Accuracy: 0.164062
Iter 300, Minibatch Loss: 2.102141, Training Accuracy: 0.171875
Iter 400, Minibatch Loss: 2.057165, Training Accuracy: 0.296875
Iter 500, Minibatch Loss: 1.929759, Training Accuracy: 0.296875
Iter 600, Minibatch Loss: 1.861923, Training Accuracy: 0.320312
Iter 700, Minibatch Loss: 1.955286, Training Accuracy: 0.343750
Iter 800, Minibatch Loss: 1.879047, Training Accuracy: 0.281250
Iter 900, Minibatch Loss: 1.791505, Training Accuracy: 0.367188
Iter 1000, Minibatch Loss: 1.664365, Training Accuracy: 0.359375
Iter 1100, Minibatch Loss: 1.801519, Training Accuracy: 0.367188
Iter 1200, Minibatch Loss: 1.755046, Training Accuracy: 0.437500
Iter 1300, Minibatch Loss: 1.662185, Training Accuracy: 0.414062
Iter 1400, Minibatch Loss: 1.473197, Training Accuracy: 0.515625
Iter 1500, Minibatch Loss: 1.505841, Training Accuracy: 0.476562
Iter 1600, Minibatch Loss: 1.49137

Iter 12500, Minibatch Loss: 0.643274, Training Accuracy: 0.796875
Iter 12600, Minibatch Loss: 0.381485, Training Accuracy: 0.859375
Iter 12700, Minibatch Loss: 0.406293, Training Accuracy: 0.898438
Iter 12800, Minibatch Loss: 0.325394, Training Accuracy: 0.882812
Iter 12900, Minibatch Loss: 0.435933, Training Accuracy: 0.843750
Iter 13000, Minibatch Loss: 0.375936, Training Accuracy: 0.851562
Iter 13100, Minibatch Loss: 0.406684, Training Accuracy: 0.898438
Iter 13200, Minibatch Loss: 0.456368, Training Accuracy: 0.828125
Iter 13300, Minibatch Loss: 0.525112, Training Accuracy: 0.843750
Iter 13400, Minibatch Loss: 0.389000, Training Accuracy: 0.890625
Iter 13500, Minibatch Loss: 0.519540, Training Accuracy: 0.835938
Iter 13600, Minibatch Loss: 0.366700, Training Accuracy: 0.890625
Iter 13700, Minibatch Loss: 0.430693, Training Accuracy: 0.867188
Iter 13800, Minibatch Loss: 0.264417, Training Accuracy: 0.921875
Iter 13900, Minibatch Loss: 0.439573, Training Accuracy: 0.867188
Iter 14000

Iter 24800, Minibatch Loss: 0.206541, Training Accuracy: 0.929688
Iter 24900, Minibatch Loss: 0.178119, Training Accuracy: 0.945312
Iter 25000, Minibatch Loss: 0.340244, Training Accuracy: 0.875000
========> Validation Accuarcy: 0.905400, Testing Accuarcy: 0.904700
Iter 25100, Minibatch Loss: 0.292307, Training Accuracy: 0.906250
Iter 25200, Minibatch Loss: 0.201705, Training Accuracy: 0.921875
Iter 25300, Minibatch Loss: 0.361588, Training Accuracy: 0.867188
Iter 25400, Minibatch Loss: 0.315385, Training Accuracy: 0.914062
Iter 25500, Minibatch Loss: 0.200823, Training Accuracy: 0.921875
Iter 25600, Minibatch Loss: 0.214021, Training Accuracy: 0.929688
Iter 25700, Minibatch Loss: 0.321480, Training Accuracy: 0.898438
Iter 25800, Minibatch Loss: 0.275712, Training Accuracy: 0.906250
Iter 25900, Minibatch Loss: 0.328479, Training Accuracy: 0.914062
Iter 26000, Minibatch Loss: 0.228756, Training Accuracy: 0.914062
Iter 26100, Minibatch Loss: 0.116678, Training Accuracy: 0.960938
Iter 262

Iter 37000, Minibatch Loss: 0.360373, Training Accuracy: 0.867188
Iter 37100, Minibatch Loss: 0.396344, Training Accuracy: 0.890625
Iter 37200, Minibatch Loss: 0.278989, Training Accuracy: 0.906250
Iter 37300, Minibatch Loss: 0.201067, Training Accuracy: 0.937500
Iter 37400, Minibatch Loss: 0.281370, Training Accuracy: 0.906250
Iter 37500, Minibatch Loss: 0.242923, Training Accuracy: 0.929688
Iter 37600, Minibatch Loss: 0.239631, Training Accuracy: 0.937500
Iter 37700, Minibatch Loss: 0.205103, Training Accuracy: 0.937500
Iter 37800, Minibatch Loss: 0.248270, Training Accuracy: 0.929688
Iter 37900, Minibatch Loss: 0.115223, Training Accuracy: 0.960938
Iter 38000, Minibatch Loss: 0.207170, Training Accuracy: 0.937500
Iter 38100, Minibatch Loss: 0.159263, Training Accuracy: 0.960938
Iter 38200, Minibatch Loss: 0.168450, Training Accuracy: 0.914062
Iter 38300, Minibatch Loss: 0.181149, Training Accuracy: 0.953125
Iter 38400, Minibatch Loss: 0.198623, Training Accuracy: 0.929688
Iter 38500

Iter 49300, Minibatch Loss: 0.293917, Training Accuracy: 0.914062
Iter 49400, Minibatch Loss: 0.266118, Training Accuracy: 0.906250
Iter 49500, Minibatch Loss: 0.137051, Training Accuracy: 0.960938
Iter 49600, Minibatch Loss: 0.267811, Training Accuracy: 0.937500
Iter 49700, Minibatch Loss: 0.145734, Training Accuracy: 0.945312
Iter 49800, Minibatch Loss: 0.204152, Training Accuracy: 0.937500
Iter 49900, Minibatch Loss: 0.176955, Training Accuracy: 0.953125
Iter 50000, Minibatch Loss: 0.089453, Training Accuracy: 0.984375
========> Validation Accuarcy: 0.933600, Testing Accuarcy: 0.932100
Iter 50100, Minibatch Loss: 0.215292, Training Accuracy: 0.937500
Iter 50200, Minibatch Loss: 0.253161, Training Accuracy: 0.914062
Iter 50300, Minibatch Loss: 0.210871, Training Accuracy: 0.937500
Iter 50400, Minibatch Loss: 0.156512, Training Accuracy: 0.929688
Iter 50500, Minibatch Loss: 0.089051, Training Accuracy: 0.968750
Iter 50600, Minibatch Loss: 0.096929, Training Accuracy: 0.984375
Iter 507